In [1]:
import os
import pymysql.cursors
from dotenv import load_dotenv

In [2]:
# Import .env to Jupyter Notebook
load_dotenv()

True

# Extract data from MySQL database

In [3]:
# connect to my local MySQL
connect = pymysql.connect(
    host=os.getenv("HOST"),
    port=3306,
    user=os.getenv("USER"),
    password=os.getenv("PASSWORD"),
    db=os.getenv("DATABASE"),
    cursorclass=pymysql.cursors.DictCursor
)

#### Show the tables in the training database

In [4]:
# try to access data in database
cursor = connect.cursor()
cursor.execute("SHOW TABLES;")
show_tables = cursor.fetchall()
cursor.close()

print(show_tables)

[{'Tables_in_training': 'online_retail'}, {'Tables_in_training': 'raw_online_retail'}]


#### Examine some of data in the table

In [5]:
# another way to access data in database
with connect.cursor() as cursor:
    sql = '''
        SELECT * FROM raw_online_retail LIMIT 5;
    '''
    cursor.execute(sql)
    number_of_data = cursor.fetchall()
 
print(f"The number of rows: {number_of_data}")

The number of rows: [{'InvoiceNo': '536365', 'StockCode': '85123A', 'Description': 'WHITE HANGING HEART T-LIGHT HOLDER', 'Quantity': 6, 'InvoiceDate': '1/12/2018 08:26', 'UnitPrice': 2.55, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '71053', 'Description': 'WHITE METAL LANTERN', 'Quantity': 6, 'InvoiceDate': '1/12/2018 08:26', 'UnitPrice': 3.39, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '84406B', 'Description': 'CREAM CUPID HEARTS COAT HANGER', 'Quantity': 8, 'InvoiceDate': '1/12/2018 08:26', 'UnitPrice': 2.75, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '84029G', 'Description': 'KNITTED UNION FLAG HOT WATER BOTTLE', 'Quantity': 6, 'InvoiceDate': '1/12/2018 08:26', 'UnitPrice': 3.39, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '84029E', 'Description': 'RED WOOLLY HOTTIE WHITE HEART.', 'Quantity': 6, 'In

#### Convert the raw data into the DataFrame with the Pandas library

In [6]:
import pandas as pd

with connect.cursor() as cursor:
    sql = '''
        SELECT * FROM raw_online_retail;
    '''
    cursor.execute(sql)
    number_of_data = cursor.fetchall()

retail_df = pd.DataFrame(number_of_data)

# Extraction data from REST API

In [7]:
import requests

In [8]:
# The transaction was generated at the London caused the currency is pound.
# We need to change pound to bath by referring the pound rate at the moment from conversion rate API.
url = 'https://de-training-2020-7au6fmnprq-de.a.run.app/currency_gbp/all'

response = requests.get(url)
response.status_code
pound_rate = response.json()

In [9]:
# convert json to DataFrame
convert_json = pd.DataFrame.from_dict(pound_rate)
convert_json

,Rate
2018-01-01T00:00:00.000Z,43.9991
2018-01-02T00:00:00.000Z,44.0731
2018-01-03T00:00:00.000Z,43.7025
2018-01-04T00:00:00.000Z,43.6914
2018-01-05T00:00:00.000Z,43.6490
...,...
2020-02-15T00:00:00.000Z,40.7223
2020-02-16T00:00:00.000Z,40.6612
2020-02-17T00:00:00.000Z,40.5658
2020-02-18T00:00:00.000Z,40.5582


In [10]:
# change the column name and reset index
convert_json = convert_json.reset_index().rename(columns={"index": "date"})
convert_json

,date,Rate
0,2018-01-01T00:00:00.000Z,43.9991
1,2018-01-02T00:00:00.000Z,44.0731
2,2018-01-03T00:00:00.000Z,43.7025
3,2018-01-04T00:00:00.000Z,43.6914
4,2018-01-05T00:00:00.000Z,43.6490
...,...,...
775,2020-02-15T00:00:00.000Z,40.7223
776,2020-02-16T00:00:00.000Z,40.6612
777,2020-02-17T00:00:00.000Z,40.5658
778,2020-02-18T00:00:00.000Z,40.5582


# Join the data between a retail data from the database 
# and a conversion rate from the REST API.

In [11]:
# create new column timestamp to keep the timestamp value before 
# change the timestamp value to date value in "InvoiceDate" column
retail_df["InvoiceTimestamp"] = retail_df["InvoiceDate"]
retail_df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/2018 08:26,2.55,17850.0,United Kingdom,1/12/2018 08:26
1,536365,71053,WHITE METAL LANTERN,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/2018 08:26,2.75,17850.0,United Kingdom,1/12/2018 08:26
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/2018 08:26,3.39,17850.0,United Kingdom,1/12/2018 08:26


In [12]:
# convert timestamp to date in both of retail_df and convert_json for joining the column
retail_df["InvoiceDate"] = pd.to_datetime(retail_df["InvoiceDate"]).dt.date
convert_json["date"] = pd.to_datetime(convert_json["date"]).dt.date
convert_json.head(3)

,date,Rate
0,2018-01-01,43.9991
1,2018-01-02,44.0731
2,2018-01-03,43.7025


In [13]:
# merge data between retail_df and conver_json by a left-join process with a key that refers to "InvoiceDate" and "date"
final_retail = retail_df.merge(convert_json, how="left", left_on="InvoiceDate", right_on="date")
final_retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2018-01-12,2.55,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
1,536365,71053,WHITE METAL LANTERN,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2018-01-12,2.75,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2018-01-12,3.39,17850.0,United Kingdom,1/12/2018 08:26,2018-01-12,43.7705
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2019-09-12,0.85,12680.0,France,9/12/2019 12:50,2019-09-12,37.5972
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2019-09-12,2.10,12680.0,France,9/12/2019 12:50,2019-09-12,37.5972
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2019-09-12,4.15,12680.0,France,9/12/2019 12:50,2019-09-12,37.5972
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2019-09-12,4.15,12680.0,France,9/12/2019 12:50,2019-09-12,37.5972


In [14]:
# create the project's purpose which is a Thai baht column by multiple between "UnitPrice" and "Rate" to "THprice"
final_retail["THprice"] = final_retail.apply(lambda x: round(x["UnitPrice"] * x["Rate"], 2), axis=1) 
final_retail.sample(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceTimestamp,date,Rate,THprice
69759,541985,22801,ANTIQUE GLASS PEDESTAL BOWL,2,2019-01-24,3.75,15052.0,United Kingdom,24/1/2019 15:52,2019-01-24,41.5907,155.97
31350,538942,22835,HOT WATER BOTTLE I AM SO POORLY,2,2018-12-15,4.65,17346.0,United Kingdom,15/12/2018 11:14,2018-12-15,41.3215,192.14
375163,569417,22196,SMALL HEART MEASURING SPOONS,24,2019-04-10,0.85,17629.0,United Kingdom,4/10/2019 10:29,2019-04-10,41.5639,35.33


# Export the final data to CSV file

In [15]:
final_retail.to_csv("{}/raw_online_retail.csv".format(os.getenv('FOLDER_PATH')))